In [3]:
#get anndata object from util file 
import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import squidpy as sq
import numpy as np
import matplotlib.pyplot as plt
import skimage
import anndata as ad
import pandas as pd
import numba
from numba import jit

In [4]:
def return_anndata(path):
    
    img = skimage.io.imread(path)
    ski_img = skimage.measure.regionprops(img)
    df = pd.DataFrame(ski_img)
    var_array = df.head(1).values
    list_var = list(var_array.flatten())
    labels = len(ski_img)
    df = pd.DataFrame(index = list_var, columns = np.arange(1,labels,1))

    
    for ind in df.index:
        for val in df.columns:
            df.loc[ind,val] = getattr(ski_img[val],ind)
            
    for val in df.columns:
        df.loc['bbox',val] = np.asarray(df.loc['bbox',val]).reshape(1,4)
        
        
    for val in df.columns:
        df.loc['centroid',val] = np.asarray(df.loc['centroid',val]).reshape(1,2)
        
        
    for ind in df.index:
        for val in df.columns:
            if type(df.loc[ind,val]) == np.ndarray:
                df.loc[ind,val] = df.loc[ind,val][~np.isnan(df.loc[ind,val])].flatten()
                
                
                
    scalar_list = ['area','area_bbox','area_convex','area_filled','axis_major_length','axis_minor_length',
            'eccentricity', 'equivalent_diameter_area','euler_number','extent','feret_diameter_max',
              'label','orientation','perimeter','perimeter_crofton','solidity'] 
    scalar_df = df.loc[['area','area_bbox','area_convex','area_filled','axis_major_length','axis_minor_length',
            'eccentricity', 'equivalent_diameter_area','euler_number','extent','feret_diameter_max',
              'label','orientation','perimeter','perimeter_crofton','solidity']]
                   
    multidim_list = ['bbox','centroid','centroid_local', 'coords', 'image', 'image_convex', 'image_filled',
                'inertia_tensor', 'inertia_tensor_eigvals', 'moments', 'moments_central' ,'moments_hu',
                'moments_normalized']
    multidim_df = df.loc[['bbox','centroid','centroid_local', 'coords', 'image', 'image_convex', 'image_filled',
                'inertia_tensor', 'inertia_tensor_eigvals', 'moments', 'moments_central' ,'moments_hu',
                'moments_normalized']]
    
    

    mt = multidim_df.transpose()
    
    X = scalar_df.T.values
    adata = ad.AnnData(X)
    adata.var_names = scalar_df.index
    adata.obs_names = scalar_df.columns
    adata.var = pd.DataFrame(index = scalar_list, columns = scalar_list)
    adata.obs = pd.DataFrame(index = scalar_df.columns, columns = scalar_df.columns)
    for col in mt.columns:
        adata.obsm[col] = mt[col].values
        
    i = 0
    for row in multidim_list:
        adata.uns[row] = multidim_df.iloc[[i]].values.flatten()
        i = i+1
        
    return adata
    

In [5]:
#loading dictionary 
import pickle 
with open('mean_intensities.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

In [6]:
def del_nan(arr):
    del_list = []
    mask = np.isnan(arr)
    for m,v in zip(mask, range(len(mask))):
        if m == True:
            del_list.append(v)
        
    arr = np.delete(arr,del_list)
    return(arr)
    
    

In [7]:
for key in loaded_dict.keys():
    val = loaded_dict[key]
    val = del_nan(val)
    #val = np.roll(val,-1)
    loaded_dict.update({key:val[1:]})
        

In [9]:
ad_object = ad_object  = return_anndata('/gpfs/commons/groups/nygcfaculty/PySeq/20210428_mouse_genotype_2/segmented_sections/m387ntga2_labels.tiff')

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/anndata/_core/anndata.py:875: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [1, 2, 3, 4, 5]

    Inferred to be: integer

  names = self._prep_dim_index(names, "obs")
/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/anndata/_core/anndata.py:801: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [1, 2, 3, 4, 5]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


In [10]:
ad_object.uns

OverloadedDict, wrapping:
	OrderedDict([('bbox', array([array([433, 665, 452, 674]), array([451, 666, 465, 675]),
       array([3114, 1425, 3128, 1440]), ...,
       array([8123, 8980, 8147, 9016]), array([8181, 9044, 8199, 9069]),
       array([10056, 10129, 10066, 10140])], dtype=object)), ('centroid', array([array([443.78181818, 668.97272727]),
       array([456.67073171, 670.15853659]),
       array([3120.46103896, 1432.35714286]), ...,
       array([8135.63309353, 9001.00719424]),
       array([8189.4375    , 9056.67857143]),
       array([10060.64556962, 10134.12658228])], dtype=object)), ('centroid_local', array([array([10.78181818,  3.97272727]), array([5.67073171, 4.15853659]),
       array([6.46103896, 7.35714286]), ...,
       array([12.63309353, 21.00719424]),
       array([ 8.4375    , 12.67857143]), array([4.64556962, 5.12658228])],
      dtype=object)), ('coords', array([array([433, 669, 433, 670, 433, 671, 434, 669, 434, 670, 434, 671, 435,
              669, 435, 670, 

In [11]:
mat = ad_object.X

In [12]:
for key in loaded_dict.keys():
    mat = np.append(mat, np.asarray([loaded_dict[key]]).transpose(), axis = 1)
    

In [13]:
#remake new anndata object
adt = ad.AnnData(mat)

In [14]:
lis = ad_object.var_names
var = list(lis)
post_var = list(loaded_dict.keys())
var.extend(post_var)

In [20]:
adt.var_names = var
adt.obs_names = np.arange(1,np.shape(mat)[0] + 1,1)
#adt.obsm = ad_object.obsm
adt.uns = ad_object.uns

In [21]:
for j in adt.uns.keys():
    #adt.obsm[j] = np.array2string(adt.obsm[j])
    adt.uns[j] = np.array2string(adt.uns[j])
    

In [22]:
adt.obs.columns = adt.obs.columns.astype(str)
adt.var.columns = adt.var.columns.astype(str)
adt.write('/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/m387ntga2_labels_anndata_sample.h5ad')

In [120]:
adt.uns['bbox'] = np.array2string(adt.uns['bbox'])

In [130]:
add = ad.read_h5ad('/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/m387ntga2_labels_anndata_sample.h5ad')

/gpfs/commons/home/jsingh/.conda/envs/spatial/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [133]:
adt.write_zarr('/gpfs/commons/groups/nygcfaculty/PySeq/spatial_analysis/m387ntga2_labels_anndata_sample.zarr')

In [134]:
adt

AnnData object with n_obs × n_vars = 11169 × 32
    uns: 'bbox', 'centroid', 'centroid_local', 'coords', 'image', 'image_convex', 'image_filled', 'inertia_tensor', 'inertia_tensor_eigvals', 'moments', 'moments_central', 'moments_hu', 'moments_normalized'

In [19]:
######### ROUGH WORK #####
for j in adt.uns.keys():
    #adt.obsm[j] = np.array2string(adt.obsm[j])
    adt.uns[j] = adt.uns[j].tobytes()
    

AttributeError: 'bytes' object has no attribute 'tobytes'